In [0]:
-- 1. Data Engineers (Full Control)
-- 2. Analysts (Read-only on gold_dlt)
-- 3. regional-manager (Restricted)
-- 4. Executives (Aggregated View)

In [0]:
-- 1. Create a dedicated schema for security logic
CREATE SCHEMA IF NOT EXISTS maven_uc.governance;

-- 2. Create the Mapping Table
-- This table links a specific person (email) to a specific region ID.
CREATE OR REPLACE TABLE maven_uc.governance.region_manager_mapping (
    manager_email STRING,
    region_id INT
);

-- 3. Configure Assignments (CRITICAL STEP)
-- Replace the emails below with the ACTUAL emails of the 4 users you created.
-- We map them to valid Region IDs from the list (26, 40, 8, 32).

TRUNCATE TABLE maven_uc.governance.region_manager_mapping;

INSERT INTO maven_uc.governance.region_manager_mapping VALUES 
  ('neha@group2ceeroutlook.onmicrosoft.com',  28),
  ('ramesh@group2ceeroutlook.onmicrosoft.com', 32);

# Row-Level Security (RLS) Filter

In [0]:
CREATE OR REPLACE VIEW maven_uc.governance.v_fact_sales AS
SELECT 
    
    --f.transaction_date,
    f.store_id,
    f.region_id,
    f.order_id,
    f.customer_id,
    f.product_id,
    f.quantity,
    f.sales_channel,
    f.sales_date
FROM maven_uc.gold_dlt.fact_sales f
JOIN maven_uc.gold_dlt.dim_stores s ON f.store_id = s.store_id
WHERE 
  -- 1. Admin Check (See All)
  is_account_group_member('data-engineer') 
  --OR is_account_group_member('Executives')
  
  OR 
  
  -- 2. Manager Check (Dynamic Row Filter)
  (
    is_account_group_member('regional-manager') 
    AND s.region_id IN (
        SELECT region_id 
        FROM maven_uc.governance.region_manager_mapping 
        WHERE manager_email = current_user()
    )
  );

In [0]:
CREATE OR REPLACE VIEW maven_uc.governance.v_dim_regions AS
SELECT * FROM maven_uc.gold_dlt.dim_regions
WHERE 
  -- Admin / Exec Check
  is_account_group_member('data-engineer') 
  OR is_account_group_member('executives')
  
  OR 
  
  -- Manager Check
  region_id IN (
      SELECT region_id 
      FROM maven_uc.governance.region_manager_mapping 
      WHERE manager_email = current_user()
  );

# Column-Level Security (Masking) Function

In [0]:
CREATE OR REPLACE VIEW maven_uc.governance.v_dim_customer AS
SELECT 
    customer_id,
    first_name,
    
    -- DYNAMIC COLUMN LOGIC
    CASE 
      WHEN is_account_group_member('data-engineer') THEN last_name
      ELSE '***** (Redacted)' 
    END AS last_name,
    CASE 
      WHEN is_account_group_member('data-engineer') THEN yearly_income
      ELSE '***** (Redacted)'
    END AS yearly_income,
    gender,
    customer_city,
    customer_country
FROM maven_uc.gold_dlt.dim_customers;

In [0]:
-- 1. REVOKE access to physical tables (Lock the Vault)
REVOKE SELECT ON TABLE maven_uc.gold_dlt.dim_customers FROM `regional-manager`;
REVOKE SELECT ON TABLE maven_uc.gold_dlt.fact_sales FROM `regional-manager`;
REVOKE SELECT ON TABLE maven_uc.gold_dlt.dim_regions FROM `regional-manager`;

-- 2. GRANT access to Dynamic Views (Give the Key to the Window)
GRANT SELECT ON VIEW maven_uc.governance.v_dim_customer TO `regional-manager`;
GRANT SELECT ON VIEW maven_uc.governance.v_fact_sales TO `regional-manager`;
GRANT SELECT ON VIEW maven_uc.governance.v_dim_regions TO `regional-manager`;

-- 3. Repeat for Analysts (Read-Only on Views)
GRANT SELECT ON VIEW maven_uc.governance.v_dim_customer TO `data-analysts`;
GRANT SELECT ON VIEW maven_uc.governance.v_fact_sales TO `data-analysts`;

# Dashboard Queries

In [0]:
-- Masking Audit (CLS):
SELECT 
    customer_id,
    gender,
    customer_city,
    customer_country,
    first_name,
    last_name,      -- Will show ***** if not Data Engineer
    yearly_income   -- Will show ***** if not Data Engineer
FROM maven_uc.governance.v_dim_customer
LIMIT 20;

In [0]:
-- Prove that access is being logged
SELECT 
    event_time, 
    user_identity.email, 
    action_name,
    request_params.table_full_name 
FROM system.access.audit 
WHERE service_name = 'unityCatalog' 
AND request_params.table_full_name LIKE 'maven_uc.gold_dlt%'
ORDER BY event_time DESC;